In [2]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from PIL import Image
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

##驗證是否安裝tensorflow
import tensorflow as tf
print(tf.__version__)
from PIL import Image
print(Image.__version__)

## 資料目錄來源 以資料夾當作分類 , e.g. Datasets/cats, Datasets/dogs, ...
src_dir = r'Datasets/'
## 單次預測圖片
predict_img = r'Datasets/test1.png'

## 影像讀取處理
datagen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(src_dir, target_size=(224, 224), batch_size=20, subset='training')
valid_generator = datagen.flow_from_directory(src_dir, target_size=(224, 224), batch_size=20, subset='validation')

# 初始化 MobileNetV2，明確指定輸入尺寸
mobilenetV2 = MobileNetV2(include_top=False, pooling='avg', input_shape=(224, 224, 3))

# 設定早期停止監控器
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',  # 監控驗證集上的損失
    patience=3,          # 在3個epoch内如果性能沒有提升，則停止
    verbose=1,
    restore_best_weights=True  # 恢復到最佳模型權重
)

## 模型建立
mobilenetV2 = MobileNetV2(include_top=False, pooling='avg')
for mlayer in mobilenetV2.layers:
    mlayer.trainable = False
mobilenetV2output = mobilenetV2.layers[-1].output
fc = layers.Dense(units=train_generator.num_classes, activation='softmax', name='custom_fc') (mobilenetV2output)
classification_model = Model(
            inputs=mobilenetV2.inputs,
            outputs=fc)
classification_model.compile(loss='categorical_crossentropy', optimizer='Adam')

## 模型訓練
history = classification_model.fit(
    train_generator, 
    epochs=50, 
    validation_data=valid_generator,
    callbacks = [early_stopping_monitor] #新增早期停止callbacks
)

## 單次預測
true_labels_dict = {}
for key in train_generator.class_indices:
    true_labels_dict[train_generator.class_indices[key]] = key

def pred(img_path):
    img = preprocess_input(np.array(Image.open(img_path).convert('RGB')))
    img = np.array([img])
    result_prob = classification_model.predict(img).tolist()[0]
    max_index = result_prob.index(max(result_prob))
    print(true_labels_dict[max_index], result_prob[max_index])

pred(predict_img)

2.16.1
10.3.0
Found 6404 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


C:\Users\owner\AppData\Local\Temp\ipykernel_19460\2344927676.py:37: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetV2 = MobileNetV2(include_top=False, pooling='avg')


Epoch 1/50


C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


321/321 ━━━━━━━━━━━━━━━━━━━━ 88s 260ms/step - loss: 0.1182 - val_loss: 0.0466
Epoch 2/50
321/321 ━━━━━━━━━━━━━━━━━━━━ 77s 241ms/step - loss: 0.0384 - val_loss: 0.0525
Epoch 3/50
321/321 ━━━━━━━━━━━━━━━━━━━━ 77s 241ms/step - loss: 0.0324 - val_loss: 0.0451
Epoch 4/50
321/321 ━━━━━━━━━━━━━━━━━━━━ 78s 244ms/step - loss: 0.0207 - val_loss: 0.0454
Epoch 5/50
321/321 ━━━━━━━━━━━━━━━━━━━━ 77s 240ms/step - loss: 0.0160 - val_loss: 0.0460
Epoch 6/50
321/321 ━━━━━━━━━━━━━━━━━━━━ 78s 244ms/step - loss: 0.0145 - val_loss: 0.0484
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 3.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 987ms/step
cats 0.9997395873069763
